In [ ]:
# This program produces the data for the phase diagram and outputs it in XLSX format.
# The data is plotted in the "Figure_4.jl" file.
# This code is large and is intended to be run on a virtual machine.

In [ ]:
using CSV
using Distributions
using LsqFit
using Plots
using PrettyTables
using Random
using Statistics
using XLSX
using Test

default(fmt = :png)

In [ ]:
function simpler_jump(x, y, a = 1)
    theta = rand(Uniform(0, 2 * π))
    u = rand(Uniform(0, 1))

    r = a/(3 * u)^(1/3)

    xfl = r * sin(theta)
    yfl = r * cos(theta)

    (floor(Int, x + xfl + 1/2),
    floor(Int, y + yfl + 1/2), r)
end

function gridoffsetandsize(infectors, removed)
    xoffset = yoffset = 0

    data = vcat(infectors, removed)

    xcoords = map(x -> x[1], data)
    ycoords = map(x -> x[2], data)

    (xmin, xmax) = (minimum(xcoords), maximum(xcoords))
    (ymin, ymax) = (minimum(ycoords), maximum(ycoords))

    if xmin < 1
        xoffset = abs(xmin)
    end

    if ymin < 1
        yoffset = abs(ymin)
    end

    gridsize = max(xmax + xoffset, ymax + yoffset)

    # add 1 because you need to include the point 0
    (xoffset + 1, yoffset + 1, gridsize + 1)
end

function creategrid(infectors, removed, xoffset, yoffset, gridsize)
    grid = zeros(gridsize, gridsize)

    infector_coords = map(x -> x[1:2], infectors)
    removed_coords = map(x -> x[1:2], removed)

    for (x, y) in removed_coords
        grid[x + xoffset, y + yoffset] = 0.5
    end

    for (x, y) in infector_coords
        grid[x + xoffset, y + yoffset] = 1
    end

    grid
    end

    function visualizegrid(grid, time)
        if 1 in grid
            colorgradient = cgrad([:black, :white, :red])
        else
            colorgradient = cgrad([:black, :white])
    end

    heatmap(1:size(grid, 1),
            1:size(grid, 2),
            grid,
            legend = :none,
            title="Time step $(time).",
            c = colorgradient,
            size=(650, 600))
end


function outbreakwithjumps_varp(maxtime :: Int,
                           maxtau :: Int,
                           nextstepfcn :: Function,
                           probinfected :: Function,
                           p_arr :: Array{Float64, 1},
                           graphcapturetimes :: Array{Int, 1},
                           addinfectors = true) :: Tuple{Array{Int, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Array{Int}, 1},
                                                   Array{Any, 1},
                                                   Array{Int64, 1}}
    # Tracking the different agents
    x_start = 0
    y_start = 0

    newinfections = Array{Int64, 1}()
    infectors = [[x_start, y_start, 0, 0]]
    numinfected = [length(infectors)]
    removed = Set([[x_start, y_start]])
    r_inf = Array{Tuple{Int64, Int64}, 1}()
    graphcomponents = Array{Any, 1}()


    for time in 1:maxtime
        ninfectors = length(infectors)
        ninfected = 0
        newinfections_now = 0

        for i in 1:ninfectors
            infector = infectors[i]
            (infector[1], infector[2]) = simpler_jump(infector[1], infector[2])
            infector[3] += 1

            inremoved = infector[1:2] in removed

            isinfecting = rand() <= probinfected(infector[1], infector[2], p_arr[1], p_arr[2], p_arr[3])

            if !inremoved && isinfecting
                push!(removed, infector[1:2])
            end

            if !inremoved && isinfecting
                newinfections_now += 1
                if !addinfectors
                    ninfected += 1
                else
                    push!(infectors, [infector[1:2]..., 0, 0])
            end

                infector[4] += 1
            end
        end

        push!(newinfections, newinfections_now)

        infectedtoremove = filter(x -> x[3] >= maxtau, infectors)
        for infector in infectedtoremove
            push!(r_inf, (infector[4], time))
        end

        filter!(x -> x[3] < maxtau, infectors)

        if addinfectors
            push!(numinfected, length(infectors))
        else
            push!(numinfected, ninfected)
        end

        if time in graphcapturetimes
            push!(graphcomponents, [copy(time), deepcopy(infectors), collect(removed)])
        end
    end

    removed = collect(removed)
#=
    comparison of the old code (first two lines) and the new one which calculated the gridsize at t = 180
    (second two lines). the important result is that there were occasionally different results which led to
    the bounds error.
(xoffset, yoffset, gridsize) = reduce(map(x -> gridoffsetandsize(x[2:end]...), graphcomponents)) do x, y
        x[3] > y[3] ? x : y
    end

(_, _, gridsizeat180) = gridoffsetandsize(graphcomponents[5][2], graphcomponents[5][3])

(gridsizeat180, gridsize)=#


    graphs = Array{Any, 1}()
    for i in 1:length(graphcomponents)
        (xoffset, yoffset, gridsize) = gridoffsetandsize(graphcomponents[i][2], graphcomponents[i][3])
        push!(graphs, creategrid(graphcomponents[i][2], graphcomponents[i][3], xoffset, yoffset, gridsize))
    end

    (numinfected, infectors, removed, graphs, newinfections)

end

function probinfected(x, y, r, p1, p2)
    if x^2 + y^2 < r^2
        return p1
    else
        return p2
    end
end

function countdecimalplaces(num)
    num = string(num)
    startcounting = false
    counter = 0;
    for i = 1:length(num)
        if startcounting
            counter += 1
        end
        if num[i] == '.'
            startcounting = true
        end
    end
    return counter
end

function predictboundaryp(τ, numdigits)
    #p = round(1.76887 * exp(-0.386626 * τ) + 0.112671, digits=numdigits)
    p = round(2.00998*τ^(-1.06421), digits=numdigits)
    return p
end

function calculateboundarypvalues(τ :: Int64, increment :: Float64)
    numdigits = countdecimalplaces(increment)
    predicted_p = predictboundaryp(τ, numdigits)
    if predicted_p > 1
        return []
    end
    if predicted_p > 0.25
        Δp = (0.01/increment)*11
    elseif predicted_p > 0.15
        Δp = (0.01/increment)*9
    else
        Δp = (0.01/increment)*9
    end
    boundary_p = Array{Float64,1}()
    for i = 1:Δp
        push!(boundary_p, predicted_p - increment*(Δp-1)/2 + increment*i)
    end

    return boundary_p
end

function boundaryphasediagram(timefunction :: Function,
                           outbreakfcn :: Function,
                           nextstepfcn :: Function,
                           probinfected :: Function,
                           p_step,
                           p_arr,
                           τ_list, #  :: UnitRange{Int64}
                           n_iterations :: Int64,
                           cutoff :: Int64)

    # times = map(x -> x, 20:40:time_input)
    times = map(x -> x, 0:5:10)
    counter = 0

    data = Array{Array{Float64,1},1}()
    boundarypvalues = Array{Array{Float64,1},1}()

    for i in 1:length(τ_list)
        push!(boundarypvalues, calculateboundarypvalues(τ_list[i], p_step))
    end

    progress = 0
    for k in 1:length(τ_list)
        maxtime = timefunction(τ_list[k])
        for p in boundarypvalues[k]
            counter = 0
            if p + p_arr[2] <= 1
                p_input = [p_arr[1], p, p + p_arr[2]]
            else
                p_input = [p_arr[1], p, 1]
            end
            @Threads.threads for i in 1:n_iterations # Hyperthreading

                (_, _, _, _, newinfections) =
                    outbreakfcn(maxtime, τ_list[k], nextstepfcn, probinfected, p_input, times)

                if last(newinfections) <= cutoff
                    counter += 1
                end
            end
            push!(data, [p, τ_list[k], counter/n_iterations])
        end

        progress += 1
        percent = 100 * progress / length(τ_list)
        if percent % 25 == 0
            println(percent, "% done.")
        end
    end

    min_p = 1;
    max_p = 0;
    for i = 1:length(data)
        if data[i][1] < min_p
            min_p = data[i][1]
        end
        if data[i][1] > max_p
            max_p = data[i][1]
        end
    end

    grid1 = zeros(Float64, floor(Int, (1/p_step)*(max_p-min_p) + 1), length(τ_list))
    for i in 1:length(data)
        x = round(Int64, (1/p_step)*(data[i][1] - min_p) + 1)
        y = convert(Int64, data[i][2] - minimum(τ_list) + 1)
        grid1[x, y] = data[i][3]
    end

    min_paxis = round(min_p / p_step) * p_step
    max_paxis = round(max_p / p_step) * p_step
    p_axis = min_paxis:p_step:max_paxis

    for c = 1:size(grid1, 2) # tau
        for r = 1:size(grid1, 1) # p
            Δp_array = calculateboundarypvalues(τ_list[c], p_step)
            if !isempty(Δp_array)
                min_Δp = minimum(Δp_array); max_Δp = maximum(Δp_array);

            else
                min_Δp = max_paxis; max_Δp = 0
            end
            predictedp = predictboundaryp(c, countdecimalplaces(p_step))
            if p_axis[r] < min_Δp
                #println(p_axis[r], "< ", min_Δp)
                grid1[r, c] = 1
            end
        end
    end

    h1 = heatmap(p_axis,
        τ_list,
        grid1,
        title="Phase Diagram",
        xguide="p", yguide="τ",
        c=cgrad([:red, :orange]),
        transpose = true,
        size=(600, 300))

    (h1, data, grid1, p_axis, τ_list)

end

function rescalepaxis(oldpaxis, τaxis, oldgrid, pstep)
    minp = minimum(oldpaxis); maxp = maximum(oldpaxis)
    newgrid = zeros(Float64, round(Int, 1/pstep), length(τaxis) )

    for c = 1:length(τaxis)
        for r = 1:round(Int, 1/pstep)
            p = r*pstep
            #display(r)
            if p >= minp && p <= maxp
                oldgrid_x = round(Int64, (1/pstep)*(p - minp) + 1)
                #println(p)
                oldgrid_y = convert(Int64, τaxis[c] - minimum(τaxis) + 1)
                newgrid[r, c] = oldgrid[oldgrid_x, c]
                #println("newgrid[", r, ",", c, "] = oldgrid[", oldgrid_x, ",", c, "]")
            else
                numdigits = countdecimalplaces(pstep)
                predicted_p = predictboundaryp(τaxis[c], numdigits)
                if p < predicted_p
                    #println("τ:", τaxis[c], "; p: ", p, "; predicted p: ", predicted_p)
                    newgrid[r, c] = 1
                else
                    newgrid[r, c] = 0
                end
            end
        end
    end

    newpaxis = pstep:pstep:1

    h1 = heatmap(newpaxis,
                τaxis,
                newgrid,
                title="Phase Diagram",
                xguide="p", yguide="τ",
                c=cgrad([:red, :orange]),
                transpose = true,
                size=(600, 300))

    (h1, newgrid, newpaxis, τaxis)
end

In [ ]:
#------------------------------------------------------------------------------
# CHANGE MODEL INPUTS HERE
function tf(τ)
    if τ < 20
        maxtime = 200
    else
        maxtime = 10 * τ
    end
    return maxtime
end

excel_filename = "tf2_tau1-100_parallel"
p_step = 0.01
τ_list = 1:100
n_iterations = 100
cutoff = 10 # less than 15 new infections at final timestep
p_arr = [20, 0] # radius of interior region and difference in p between interior and exterior (exterior will have higher p)

t1 = time()
(plot, data, grid_data, paxis, τaxis) = boundaryphasediagram(tf, outbreakwithjumps_varp, simpler_jump, probinfected, p_step, p_arr, τ_list, n_iterations, cutoff)
t2 = time()
t_elapsed = t2-t1
println(string("Time to produce phase diagram:", t_elapsed, " seconds"))

(rescaled_plot, rescaled_grid_data, newpaxis, τaxis) = rescalepaxis(paxis, τaxis, grid_data, p_step)

#l = @layout [a; b]
#plot(plot, rescaled_plot, layout=l)


In [ ]:
# Export to XLSX

function fill_sheet1(sheet, arr)
    for ind in CartesianIndices(arr)
        XLSX.setdata!(sheet, XLSX.CellRef(ind[1], ind[2]), arr[ind])
        #println(sheet, ": ", ind[1], ", ", ind[2])

    end
end

function fill_sheet2(sheet, arr)
    # the 2 function is for a n x 1 array of arrays (each of the smaller arrays has 3 elements)
    for i in 1:length(arr)
        XLSX.setdata!(sheet, XLSX.CellRef(i, 1), arr[i][1])
        XLSX.setdata!(sheet, XLSX.CellRef(i, 2), arr[i][2])
        XLSX.setdata!(sheet, XLSX.CellRef(i, 3), arr[i][3])
    end
end

fn = string(excel_filename, ".xlsx")
XLSX.openxlsx(fn, mode="w") do xf
    s1 = xf["Sheet1"]
    XLSX.setdata!(s1, XLSX.CellRef(1, 1), "Time Elasped")
    XLSX.setdata!(s1, XLSX.CellRef(1, 2), t_elapsed)
    XLSX.setdata!(s1, XLSX.CellRef(2, 1), "Original P Axis")
    XLSX.setdata!(s1, XLSX.CellRef(2, 2), paxis[1])
    XLSX.setdata!(s1, XLSX.CellRef(2, 3), "to")
    XLSX.setdata!(s1, XLSX.CellRef(2, 4), paxis[end])
    XLSX.setdata!(s1, XLSX.CellRef(3, 1), "Rescaled P Axis")
    XLSX.setdata!(s1, XLSX.CellRef(3, 2), newpaxis[1])
    XLSX.setdata!(s1, XLSX.CellRef(3, 3), "to")
    XLSX.setdata!(s1, XLSX.CellRef(3, 4), newpaxis[end])
    XLSX.setdata!(s1, XLSX.CellRef(4, 1), "Original Tau Axis")
    XLSX.setdata!(s1, XLSX.CellRef(4, 2), τaxis[1])
    XLSX.setdata!(s1, XLSX.CellRef(4, 3), "to")
    XLSX.setdata!(s1, XLSX.CellRef(4, 4), τaxis[end])
    XLSX.setdata!(s1, XLSX.CellRef(5, 1), "New Tau Axis")
    XLSX.setdata!(s1, XLSX.CellRef(5, 2), τaxis[1])
    XLSX.setdata!(s1, XLSX.CellRef(5, 3), "to")
    XLSX.setdata!(s1, XLSX.CellRef(5, 4), τaxis[end])

    XLSX.setdata!(s1, XLSX.CellRef(7, 1), "P Step")
    XLSX.setdata!(s1, XLSX.CellRef(7, 2), p_step)
    XLSX.setdata!(s1, XLSX.CellRef(8, 1), "N Iterations")
    XLSX.setdata!(s1, XLSX.CellRef(8, 2), n_iterations)
    XLSX.setdata!(s1, XLSX.CellRef(9, 1), "Cutoff")
    XLSX.setdata!(s1, XLSX.CellRef(9, 2), cutoff)
    XLSX.setdata!(s1, XLSX.CellRef(10, 1), "Inner region radius")
    XLSX.setdata!(s1, XLSX.CellRef(10, 2), p_arr[1])
    XLSX.setdata!(s1, XLSX.CellRef(11, 1), "P_out - P_ins")
    XLSX.setdata!(s1, XLSX.CellRef(11, 2), p_arr[2])

    s2  = XLSX.addsheet!(xf,"Grid Data")
    fill_sheet1(s2,grid_data)
    println("Exported Grid Data to Excel!")

    s3 = XLSX.addsheet!(xf,"Rescaled Grid Data")
    fill_sheet1(s3,rescaled_grid_data)
    println("Exported Rescaled Grid Data to Excel!")

    s4 = XLSX.addsheet!(xf, "Raw Data")
    fill_sheet2(s4, data)
    println("Exported Raw Data to Excel!")
end